In [1]:
#准备数据集和模型建立
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
# 超参数
learning_rate = 1e-3 #学习率，每个纪元更新模型参数的数量。较小的值会导致学习速度变慢，而较大的值可能会导致训练期间不可预测的行为。
batch_size = 64 #批量大小，更新参数前通过网络传播的样本数
epochs = 5 #纪元数，迭代数据集的次数

In [3]:
# 优化循环
# 训练循环，遍历训练数据集并尝试收敛到最佳参数。
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 每次迭代中，计算预测值与损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 优化过程，收集误差相对于其参数的导数
        optimizer.zero_grad() #重置模型参数梯度，防止重复计算
        loss.backward() #反向传播预测损失，获得损失梯度
        optimizer.step() #利用损失梯度调整参数

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 测试循环，循环测试数据集以检查模型性能是否正在提高。
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:
# 损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
# 使用梯度下降，优化调整模型参数减少模型误差
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 训练开始
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305083  [    0/60000]
loss: 2.290298  [ 6400/60000]
loss: 2.269175  [12800/60000]
loss: 2.259341  [19200/60000]
loss: 2.254297  [25600/60000]
loss: 2.219677  [32000/60000]
loss: 2.220700  [38400/60000]
loss: 2.195437  [44800/60000]
loss: 2.184643  [51200/60000]
loss: 2.152946  [57600/60000]
Test Error: 
 Accuracy: 51.3%, Avg loss: 2.150762 

Epoch 2
-------------------------------
loss: 2.159744  [    0/60000]
loss: 2.148659  [ 6400/60000]
loss: 2.089580  [12800/60000]
loss: 2.099488  [19200/60000]
loss: 2.069659  [25600/60000]
loss: 1.998757  [32000/60000]
loss: 2.014333  [38400/60000]
loss: 1.945849  [44800/60000]
loss: 1.943166  [51200/60000]
loss: 1.864640  [57600/60000]
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.871620 

Epoch 3
-------------------------------
loss: 1.906067  [    0/60000]
loss: 1.870577  [ 6400/60000]
loss: 1.757369  [12800/60000]
loss: 1.785101  [19200/60000]
loss: 1.707888  [25600/60000]
loss: 1.648299  [32000/600